<a href="https://colab.research.google.com/github/abxda/Taller_UAEMEX_2025/blob/main/Taller_UAEMEX_02.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install --upgrade pycaret

In [ ]:
#REINICIAR SESION

In [ ]:
import pycaret
from pycaret.classification import *
import pandas as pd
import numpy as np

In [ ]:
csv_file = 'features_sentinel_urbano.csv'
print(f"\nCargando datos desde: {csv_file}")
data = pd.read_csv(csv_file)
print(f"Datos cargados: {data.shape[0]} filas, {data.shape[1]} columnas")
print("Primeras filas:")
print(data.head())
print("\nNombres de columnas:")
print(data.columns.tolist())

In [ ]:
id_column_original_name = data.columns[0] # Asumir que el ID está en la primera columna
id_column_new_name = 'FID' # Nombre estándar para el ID
data.rename(columns={id_column_original_name: id_column_new_name}, inplace=True)
print(f"\nColumna de ID renombrada de '{id_column_original_name}' a '{id_column_new_name}'")


In [ ]:
print("\nAplicando filtro de pureza >= 90% y balanceo (similar al tutorial original)...")
# Asumiendo klass=1 (Urbano), klass=2 (No Urbano) como vienen de RSGISLib
urbanas_puras = data[(data['klass'] == 1) & (data['klass__1'] >= 0.9)]
no_urbanas_puras = data[(data['klass'] == 2) & (data['klass__2'] >= 0.9)]
print(f"  Celdas urbanas puras (>=90%): {len(urbanas_puras)}")
print(f"  Celdas no urbanas puras (>=90%): {len(no_urbanas_puras)}")

In [ ]:
# Balancear muestreando la clase mayor para igualar la menor
n_min = min(len(urbanas_puras), len(no_urbanas_puras))
df1 = urbanas_puras.sample(n=n_min, random_state=123)
df2 = no_urbanas_puras.sample(n=n_min, random_state=123)
data_balanced = pd.concat([df1, df2], ignore_index=True)
print(f"  Dataset balanceado creado con {len(data_balanced)} filas ({n_min} de cada clase pura).")

In [ ]:
print("\nRemapeando la columna 'klass': 2 se convierte en 0 (No Urbano)")
data_balanced['klass'] = data_balanced['klass'].replace({2: 0})
print("Distribución de la clase 'klass' después del remapeo:")
print(data_balanced['klass'].value_counts())

In [ ]:
target_column = 'klass'
columns_to_ignore = [id_column_new_name, 'klass__1', 'klass__2']
# Construir la lista de features automáticamente
all_columns = data_balanced.columns.tolist()
feature_columns = [
    col for col in all_columns
    if col != target_column and col not in columns_to_ignore
]
# Verificar que las features son las esperadas (deberían ser las 60 estadísticas)
print(f"\nSe usarán {len(feature_columns)} columnas como features.")
# print("Features seleccionadas:", feature_columns) # Descomentar para ver la lista

# Crear el DataFrame final para PyCaret (solo target y features)
data_for_pycaret = data_balanced[[target_column] + feature_columns]
print(f"Dimensiones del DataFrame final para PyCaret: {data_for_pycaret.shape}")


In [ ]:
data_for_pycaret

In [ ]:
s = setup(data_for_pycaret,
          target = target_column,
          session_id = 123,  # Para reproducibilidad
          log_experiment = False # Evitar logging a MLflow/WandB por simplicidad aquí
          )

In [ ]:
best = compare_models(sort = 'Accuracy', exclude=['gbc'])

In [ ]:
best

In [ ]:
print("\nParámetros del mejor modelo:")
print(best.get_params())
model_filename = 'modelo_pycaret_clasificacion_urbana'
print(f"\nGuardando el pipeline del modelo como '{model_filename}.pkl'")
save_model(best, model_filename)

In [ ]:
evaluate_model(best)

**1. ¿Qué Muestra la Gráfica?**

Esta gráfica resume dos cosas clave:

* **Importancia Global de las Características:** Qué tan influyente es cada característica (ej. `b2Max`, `b11Min`) en *general* para las predicciones del modelo en todo el conjunto de datos. Las características se ordenan verticalmente de mayor a menor importancia (la más importante arriba).
* **Impacto Local y Relación con el Valor:** Para cada característica, muestra cómo su valor afecta la predicción para *cada observación individual* (cada celda de 100m en nuestro caso).

**2. Componentes de la Gráfica:**

* **Eje Vertical (Características):** Aquí se listan las características de entrada que utilizaste (las estadísticas por banda como `bNMin`, `bNMax`, `bNMean`, `bNStdDev`, `bNSum`). Están ordenadas por su importancia global decreciente. La característica en la cima (`b2Max` en tu gráfica) es la que tiene el mayor impacto promedio en las predicciones del modelo.
* **Eje Horizontal (Valor SHAP):** Este eje cuantifica el **impacto** de una característica en la salida del modelo para una observación específica.
    * **SHAP = 0:** La característica no tuvo impacto en esa predicción en particular, comparado con la predicción promedio del modelo.
    * **SHAP > 0 (Valores Positivos):** La característica empujó la predicción del modelo hacia la clase positiva. En nuestro caso, como remapeamos `klass` 1 (Urbano) a 1 y `klass` 2 (No Urbano) a 0, los valores SHAP positivos empujan la predicción hacia **"Urbano" (clase 1)**.
    * **SHAP < 0 (Valores Negativos):** La característica empujó la predicción del modelo hacia la clase negativa, es decir, hacia **"No Urbano" (clase 0)**.
    Cuanto más lejos de cero esté el valor SHAP (a la derecha o izquierda), mayor fue el impacto de esa característica en esa predicción específica.
* **Puntos (Enjambre):** Cada punto en la gráfica representa una única celda de 100m de nuestro conjunto de datos de prueba (o la muestra utilizada por SHAP). Su posición horizontal indica el valor SHAP para esa celda y esa característica específica. Los puntos se dispersan verticalmente dentro de la fila de cada característica para evitar la superposición y dar una idea de la distribución de los impactos.
* **Color (Valor de la Característica):** El color de cada punto indica el **valor original** de esa característica para esa celda. La barra de color a la derecha muestra la escala: típicamente, **Rojo (High)** indica valores altos de la característica y **Azul (Low)** indica valores bajos.

**3. Interpretación Específica de tu Gráfica:**

Ahora, apliquemos esto a tu gráfica:

* **Características Más Importantes:** Las características en la parte superior son las más influyentes globalmente. En tu caso, parecen ser:
    1.  `b2Max` (Máximo de la Banda 2 - Azul)
    2.  `b1Max` (Máximo de la Banda 1 - Aerosol Costero)
    3.  `b11Max` (Máximo de la Banda 11 - SWIR 1)
    4.  `b11Min` (Mínimo de la Banda 11 - SWIR 1)
    5.  `b11Mean` (Media de la Banda 11 - SWIR 1)
    6.  `b1Min` (Mínimo de la Banda 1 - Aerosol Costero)
    Esto sugiere que las bandas Azul, Aerosol Costero y SWIR 1 (especialmente sus valores máximos y mínimos) son cruciales para que el modelo distinga entre zonas urbanas y no urbanas en Aguascalientes, usando este conjunto de datos y modelo específico.

* **¿Cómo Afectan las Características?**
    * **`b2Max` (Azul Máx):** Observa que los puntos rojos (valores altos de Azul Máx) están casi todos a la derecha (SHAP > 0), empujando la predicción hacia Urbano. Los puntos azules (valores bajos) están a la izquierda (SHAP < 0), empujando hacia No Urbano. *Interpretación: Valores máximos altos de reflectancia en la banda azul dentro de una celda de 100m son un fuerte indicador de área urbana para este modelo.*
    * **`b1Max` (Aerosol Máx):** Sigue un patrón muy similar al `b2Max`. *Interpretación: Valores máximos altos en la banda de aerosol costero también indican fuertemente áreas urbanas.*
    * **`b11Max` (SWIR 1 Máx):** Mismo patrón: rojo (alto SWIR 1 Máx) a la derecha (Urbano), azul (bajo SWIR 1 Máx) a la izquierda (No Urbano). *Interpretación: Máximos altos en SWIR 1 son característicos de zonas urbanas.*
    * **`b11Min` (SWIR 1 Mín):** Aquí el patrón se invierte interesante. Los puntos rojos (valores altos de SWIR 1 Mín) están principalmente a la izquierda (SHAP < 0, empujan a No Urbano), mientras que los puntos azules (valores bajos de SWIR 1 Mín) tienden a estar a la derecha (SHAP > 0, empujan a Urbano). *Interpretación: Valores mínimos bajos en SWIR 1 son indicativos de áreas urbanas, mientras que si el mínimo es alto, sugiere área no urbana.* Esto tiene sentido físico, ya que superficies construidas podrían tener mínimos de SWIR consistentes, mientras que la vegetación o suelos pueden tener mínimos más altos.
    * **`b11Mean` (SWIR 1 Media):** Un patrón similar a `b11Max` pero menos separado: valores altos (rojo) tienden a empujar a Urbano, valores bajos (azul) tienden a empujar a No Urbano.
    * **Otras (Ej. `b1StdDev`, `b8StdDev`):** Observa las desviaciones estándar. Para `b1StdDev` (Desv. Est. Aerosol), valores altos (rojo) tienden a tener SHAP positivo (Urbano), y valores bajos (azul) tienen SHAP negativo (No Urbano). *Interpretación: Una mayor variabilidad (desviación estándar alta) en la banda de aerosol costero dentro de la celda de 100m parece ser más común en zonas urbanas.*

**En Resumen:**

Esta gráfica SHAP te dice que tu modelo de clasificación depende fuertemente de las estadísticas de las bandas del **Azul (b2)**, **Aerosol Costero (b1)** y especialmente del **Infrarrojo de Onda Corta 1 (b11)**. En general:
* Valores **máximos altos** en estas bandas indican **Zona Urbana**.
* Valores **mínimos bajos** en **SWIR 1 (b11)** también indican **Zona Urbana**.
* La **variabilidad** (desviación estándar) en algunas bandas (como b1 y b8) también aporta información distintiva.

Te permite entender no solo *qué* características son importantes, sino *cómo* sus valores específicos (altos o bajos) influyen en la decisión del modelo para cada celda analizada. Es una ventana excelente a la "mente" de tu modelo de Machine Learning.

In [ ]:
interpret_model(best)

Hemos recorrido un largo camino, desde la preparación inicial de los datos satelitales y vectoriales hasta el entrenamiento y selección de un modelo óptimo de Machine Learning usando PyCaret. Ahora, llegamos a la culminación de nuestro esfuerzo: aplicar el modelo entrenado para generar un mapa completo que visualice la predicción de zonas urbanas para toda nuestra área de estudio en Aguascalientes.

**Propósito del Código:**

Este script final toma el modelo de clasificación que guardamos (`modelo_pycaret_clasificacion_urbana.pkl`), lo aplica al conjunto completo de características extraídas de *todas* las celdas de 100 metros (`features_sentinel_urbano.csv`), y vincula estas predicciones con las geometrías espaciales correspondientes de nuestro grid (`grid_100m_aea.gpkg`). El objetivo es producir un resultado geoespacial final, listo para ser visualizado y analizado en un Sistema de Información Geográfica (SIG).

**Proceso Clave del Código:**

1.  **Carga de Recursos Finales:** El script comienza cargando los tres componentes esenciales:
    * El pipeline completo del modelo PyCaret entrenado (`load_model`).
    * El archivo CSV que contiene las características espectrales calculadas para *cada una* de las celdas de 100m en el área de estudio.
    * El archivo GeoPackage que contiene los polígonos georreferenciados de nuestro grid de 100m, cada uno con su identificador único.
2.  **Preparación de Datos para Predicción:** Se asegura de que los datos del CSV estén listos para el modelo. Se renombra la columna de ID a 'FID' por consistencia y, lo más importante, se seleccionan únicamente las columnas que corresponden a las características (las 60 estadísticas de banda) que el modelo espera recibir, utilizando la información almacenada dentro del propio pipeline cargado (`model.named_steps['trained_model'].feature_names_in_`).
3.  **Predicción Masiva:** Se utiliza la función `predict_model` de PyCaret. Esta función aplica eficientemente todo el pipeline del modelo (incluyendo cualquier preprocesamiento interno que se haya definido en la fase de `setup`) a todas las filas del DataFrame de características. Genera un nuevo DataFrame que contiene las predicciones finales:
    * `prediction_label`: La clase predicha (0 para No Urbano, 1 para Urbano).
    * `prediction_score`: La probabilidad estimada por el modelo de que la celda pertenezca a la clase 1 (Urbano). Esta puntuación nos da una idea de la "confianza" del modelo en su predicción.
4.  **Unión Espacial:** Los resultados de la predicción (la etiqueta y la puntuación de probabilidad, junto con el FID) se unen espacialmente con el GeoDataFrame que contiene las geometrías del grid. La unión se realiza utilizando el 'FID' como clave común, asegurando que cada polígono del grid reciba la predicción y la probabilidad que le corresponden.
5.  **Guardado del Resultado Final:** El GeoDataFrame resultante, que ahora es un mapa completo con los polígonos del grid y sus atributos de predicción, se guarda como un nuevo archivo GeoPackage (`predicciones_urbanas_mapa.gpkg`). Se especifica un nombre de capa (`predicciones_urbanas`) dentro del GeoPackage.

**Producto Generado por este Bloque:**

El resultado tangible de este script es el archivo **`predicciones_urbanas_mapa.gpkg`**. Este archivo GeoPackage contiene el mapa final de nuestro análisis. Al abrirlo en un software SIG como QGIS, podrás visualizar el grid de 100 metros coloreado según la clase predicha (`prediction_label`) y también podrías tematizarlo usando la probabilidad (`prediction_score`) para explorar las áreas donde el modelo tiene mayor o menor certeza. ¡Este mapa representa el producto final de aplicar Machine Learning para detectar zonas urbanas en tus imágenes Sentinel-2!

---

In [ ]:
import pandas as pd
import geopandas as gpd
import os
from pycaret.classification import load_model, predict_model


csv_features_file = 'features_sentinel_urbano.csv'
model_base_name = 'modelo_pycaret_clasificacion_urbana' # Sin .pkl
grid_geopackage_file = 'grid_100m_aea.gpkg'
output_prediction_map_gpkg = 'predicciones_urbanas_mapa.gpkg'


try:
    # 1. Cargar modelo, datos CSV, y geometrías GPKG
    print("- Cargando recursos...")
    model = load_model(model_base_name)
    all_features_df = pd.read_csv(csv_features_file)
    grid_gdf = gpd.read_file(grid_geopackage_file) # Asume capa 'grid_100m' o única

    # 2. Preparar datos para predicción
    print("- Preparando datos para predicción...")
    # Renombrar columna ID del CSV
    id_col_original_csv = all_features_df.columns[0]
    id_col_target_csv = 'FID'
    all_features_df.rename(columns={id_col_original_csv: id_col_target_csv}, inplace=True)

    # Seleccionar solo las features que el modelo espera (excluyendo ID y otras columnas no usadas)
    # Obtenemos los nombres de las features directamente del pipeline/modelo cargado
    expected_features = model.named_steps['trained_model'].feature_names_in_
    data_to_predict = all_features_df[expected_features] # Pasar solo las features a predict_model

    # 3. Realizar predicciones
    print("- Realizando predicciones...")
    # predict_model devuelve un DataFrame con las features originales MÁS 'prediction_label' y 'prediction_score'
    predictions_df_full = predict_model(model, data=data_to_predict)

    # Seleccionar solo ID original y las predicciones para unir
    # Necesitamos añadir el FID de vuelta para la unión
    predictions_to_merge = all_features_df[[id_col_target_csv]].join(
        predictions_df_full[['prediction_label', 'prediction_score']]
    )

    # 4. Preparar GPKG y Unir resultados
    print("- Uniendo predicciones con geometrías...")
    # Renombrar columna 'id' del GeoPackage a 'FID' si es necesario para la unión
    if 'id' in grid_gdf.columns and id_col_target_csv not in grid_gdf.columns:
        grid_gdf.rename(columns={'id': id_col_target_csv}, inplace=True)

    # Unir
    map_df = grid_gdf.merge(predictions_to_merge, on=id_col_target_csv, how='left')

    # 5. Guardar mapa final
    print(f"- Guardando mapa final en: {output_prediction_map_gpkg}")
    map_df.to_file(output_prediction_map_gpkg, driver='GPKG', layer='predicciones_urbanas')

    print("\n--- Proceso completado exitosamente ---")
    print(f"Mapa de predicciones guardado como '{output_prediction_map_gpkg}'.")
    print("Columnas finales en el GeoPackage:", map_df.columns.tolist())

except FileNotFoundError as e:
     print(f"\nError: Archivo no encontrado - {e}. Asegúrate que todos los archivos de entrada existen.")
except KeyError as e:
     print(f"\nError: Problema con nombre de columna - {e}. Verifica los nombres 'FID'/'id' en tus archivos.")
except Exception as e:
    print(f"\nOcurrió un error inesperado: {e}")
    import traceback
    traceback.print_exc()

In [ ]:
from google.colab import files
files.download('predicciones_urbanas_mapa.gpkg')